In [1]:
import pandas as pd
import pickle
import numpy as np
import dataloader
import torch

In [2]:
datasets = ['Adobe240', 'UCF-101', 'Vimeo90k']
methods = ['DAIN', 'SepConv', 'QVI', 'QVI_linear']
metrics = ['psnr', 'ssim', 'ie']

## Metrics

The following metrics are reported in the benchmark, for a predicted and ground truth image frame $\hat{I}$ and $I$ respectively:

\begin{equation}
    \mathrm{SSIM}(\hat{I}, I) = 
    \frac{2 \mu_{\hat{I}} \mu_I + C_1}{\mu^2_{\hat{I}} + \mu^2_I+C_1}
    \frac{2 \sigma_{\hat{I}} \sigma_I + C_2}{\sigma^2_{\hat{I}}+\sigma_I^2+C_2}
    \frac{2 \sigma^2_{\hat{I}} + C_3}{\sigma_{\hat{I}} \sigma_I^2 + C_3}
\end{equation}

$$\mathrm{PSNR}(\hat{I}, I) = 10 \, \log_{10} \left(\frac{R^2}{||\hat{I}-I||_2} \right)$$

$$\mathrm{IE}(\hat{I}, I) = \left( 
    \frac{1}{N} \sum_x (\hat{I}_x - I_x)^2
    \right)^{1/2}$$

## Datasets

### Middlebury benchmark

Baker et al. https://ps.is.mpg.de/publications/baker-ijcv-11

This is the standard benchmarking dataset for frame interpolation. Aside from the publicly available test set, they also host an online evaluation system where you can submit predicted interpolated frames. The website will then calculate the metrics serverside. This ensures that the results are obtained honestly.

Their work is an advancement on the earlier work of Barron et al. (https://link.springer.com/article/10.1007/BF01420984 ). The resulting dataset consists of more natural and complex scenes, making it possible to use it as a real word benchmark. It does only consist of a small set of images, so it is only useful for evaluation, not for training.

### UCF101

Soomro et al. https://arxiv.org/abs/1212.0402

This dataset focusses on video clips containing 'actions'. The dataset covers 101 action classes, covering applying make up, surfing and shaving. Their work is an extension of the earlier UCF50 dataset, which contained only 50 action classes (https://www.crcv.ucf.edu/research/recognizing-50-human-action-categories-of-web-videos/ ). The authors claim that the dataset is challenging, because of its wide variety of content, and because of the changing lighting conditions and low frame rate. Despite the fact that the dataset is originally used to build models to classify the action present in the video, it is also often used for video frame interpolation.

### Vimeo90K

Xue et al. https://arxiv.org/abs/1711.09078

The Vimeo90K dataset, named after the nearly 90.000 video sequences extracted from the original videos from Vimeo, is does not comprise of videos, but is actually already preprocessed and extracted into frames directly usable for the video frame interpolation task. Each frame sequence consists of seven consecutive frames, and each frame has a resolution of $256\times 448$. This dataset is one of the first high quality benchmarking data sets for video frame interpolation, since the original videos were all originally 720p or of higher quality. To standardize the dataset, all videos were downsampled to $256\times 448$.

### Adobe240

Su et al. https://www.cs.ubc.ca/labs/imager/tr/2017/DeepVideoDeblurring/

The Adobe240 dataset contains 133 high-quality 720p videos, i.e. having a frame size of $720 \times 1280$. The dataset is named after its frame rate, all of these videos are recorded in 240 frames per second. The authors created this datases because they needed to create a method for deblurring videos on high frame rate video data. They recorded the videos themselves using a variety of devices, decreasing the bias towards one of the cameras and improve the generalizability.

## Benchmark

In [11]:
with open('results_new.pkl', 'rb') as f:
    r = pickle.load(f)

df = pd.DataFrame()

for dataset in datasets:
    for metric in metrics:
        df[f'{dataset}_{metric}'] = [np.mean(r[mtd][dataset][metric]) for mtd in methods]

df.index = methods
df

,Adobe240_psnr,Adobe240_ssim,Adobe240_ie,UCF-101_psnr,UCF-101_ssim,UCF-101_ie,Vimeo90k_psnr,Vimeo90k_ssim,Vimeo90k_ie
DAIN,32.153012,0.988339,6.987570,27.878556,0.989216,10.920331,30.567242,0.984594,7.889040
SepConv,36.155629,0.992890,4.653473,35.559125,0.996604,5.154202,32.811017,0.987650,6.429720
QVI,34.246344,0.986623,6.210768,35.735078,0.996410,5.217945,36.820013,0.998507,3.547599
QVI_linear,35.018728,0.992383,5.124491,35.436220,0.996504,5.244211,36.530777,0.998404,3.667721


## Qualitative results